In [ ]:
from google.colab import userdata
GEMINI_API_KEY = userdata.get('GEMINI')
WEATHER_API_KEY = userdata.get('WEATHER_API_KEY')

In [ ]:
# @title Installation
!pip install google-adk -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.

In [ ]:
# @title Import necessary libraries
import os
import asyncio
from google.adk.agents import Agent
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner
from google.genai import types # For creating message Content/Parts
import httpx

In [ ]:
# @title Weather Tool Definition
def get_weather(city: str) -> str:
    """Fetches current weather data for a city in India."""
    WEATHER_API_BASE = "http://api.weatherapi.com/v1"
    endpoint = "current.json"
    params = {
        "key": WEATHER_API_KEY,
        "q": f"{city},India",
        "aqi": "no"
    }

    try:
        response = httpx.get(f"{WEATHER_API_BASE}/{endpoint}", params=params, timeout=30.0)
        response.raise_for_status()
        data = response.json()
        location = data["location"]
        current = data["current"]

        return f"""
Location: {location['name']}, {location['region']}, {location['country']}
Temperature: {current['temp_c']}°C
Condition: {current['condition']['text']}
Humidity: {current['humidity']}%
Wind Speed: {current['wind_kph']} km/h
Last Updated: {current['last_updated']}
        """.strip()

    except Exception as e:
        print(f"Error fetching weather: {e}")
        return "Unable to fetch weather data. Please check the city name and try again."

In [ ]:
get_weather("Pune")

'Location: Pune, Maharashtra, India\nTemperature: 28.8°C\nCondition: Patchy rain nearby\nHumidity: 58%\nWind Speed: 6.1 km/h\nLast Updated: 2025-05-16 09:00'

In [ ]:
os.environ["GOOGLE_API_KEY"] = GEMINI_API_KEY
# @title define agent
weather_agent = Agent(
    name="weather_agent_v1",
    model="gemini-2.0-flash",
    description="Provides weather information for specific cities.",
    instruction="You are a helpful weather assistant. "
                "When the user asks for the weather in a specific city, "
                "use the 'get_weather' tool to find the information. "
                "If the tool returns an error, inform the user politely. "
                "If the tool is successful, present the weather report clearly.",
    tools=[get_weather], # Pass the function directly
)

In [ ]:
# @title Setup Session Service and Runner

#SessionService stores conversation history & state.
session_service = InMemorySessionService()

# Define constants for identifying the interaction context
APP_NAME = "weather_tutorial_app"
USER_ID = "user_1"
SESSION_ID = "session_001"

# Create the specific session where the conversation will happen
session = session_service.create_session(
    app_name=APP_NAME,
    user_id=USER_ID,
    session_id=SESSION_ID
)

# Runner orchestrates the agent execution loop.
runner = Runner(
    agent=weather_agent,
    app_name=APP_NAME,
    session_service=session_service
)

In [ ]:
# @title Define Agent Interaction Function
async def call_agent_async(query: str, runner, user_id, session_id):
  """Sends a query to the agent and prints the final response."""
  print(f"\n>>> User Query: {query}")

  content = types.Content(role='user', parts=[types.Part(text=query)])

  # run_async executes the agent logic and yields Events.
  async for event in runner.run_async(user_id=user_id, session_id=session_id, new_message=content):
      # print(f"  [Event] Author: {event.author}, Type: {type(event).__name__}, Final: {event.is_final_response()}, Content: {event.content}")

      if event.is_final_response():
          if event.content and event.content.parts:
             final_response_text = event.content.parts[0].text
          elif event.actions and event.actions.escalate: # Handle potential errors/escalations
             final_response_text = f"Agent escalated: {event.error_message or 'No specific message.'}"
          break

  print(f"<<< Agent Response: {final_response_text}")

In [ ]:
async def run_interactive_conversation():
    while True:
        user_query = input("Ask me something (or type 'exit' to quit): ")
        if user_query.lower() == 'exit':
            break
        await call_agent_async(user_query,runner=runner,user_id=USER_ID,session_id=SESSION_ID)

# Execute the interactive conversation
await run_interactive_conversation()

Ask me something (or type 'exit' to quit): what can yu do

>>> User Query: what can yu do
<<< Agent Response: I can provide weather information for cities in India. Just tell me which city you're interested in!

Ask me something (or type 'exit' to quit): i want climate of maharashtra

>>> User Query: i want climate of maharashtra
<<< Agent Response: I can only provide weather information for a specific city. Maharashtra is a state. Could you please specify a city within Maharashtra for which you want the weather? For example, you could ask about Mumbai, Pune, or Nagpur.

Ask me something (or type 'exit' to quit): Temperature of pune

>>> User Query: Temperature of pune


<<< Agent Response: OK. The temperature in Pune, Maharashtra, India is 28.8°C. The condition is Patchy rain nearby, with 58% humidity and a wind speed of 6.1 km/h. This information was last updated on 2025-05-16 at 09:00.

Ask me something (or type 'exit' to quit): exit
